In [ ]:
import numpy as np
import cv2
import hashlib
import pandas as pd

In [5]:
# CONSTANTS
IMAGE = cv2.imread("lava_lamp.jpeg")
THRESHOLD = 0.25
RESIZED_IMG = cv2.cvtColor(cv2.resize(IMAGE, (600, 600)), cv2.COLOR_BGR2RGB)
EXP_RES_IMG = RESIZED_IMG.copy()

In [83]:
def get_available_cameras():
    available_cameras = []
    # Check for 5 cameras
    for i in range(5):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            available_cameras.append(i)
            cap.release()
    return available_cameras


cameras = get_available_cameras()
if cameras:
    print("Available Cameras:", cameras)
else:
    print("No cameras found.")

Available Cameras: [0, 1, 2]


In [ ]:
def create_combined_window(frames_dict, target_width=800):
    """Combine multiple frames into a single window with resizing"""
    # Get the size of the first frame for reference
    first_frame = next(iter(frames_dict.values()))
    height, width = first_frame.shape[:2]

    # Calculate new dimensions maintaining aspect ratio
    target_height = int(target_width * height / width)

    # Create a blank canvas to hold all frames
    combined = np.zeros(
        (target_height, target_width * len(frames_dict), 3), dtype=np.uint8
    )

    # Add each frame to the canvas
    for idx, (name, frame) in enumerate(frames_dict.items()):
        # Resize frame
        resized_frame = cv2.resize(frame, (target_width, target_height))

        # Convert grayscale to BGR if needed
        if len(resized_frame.shape) == 2:
            resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_GRAY2BGR)

        # Add frame to canvas
        combined[:, idx * target_width : (idx + 1) * target_width] = resized_frame

        # Add label
        cv2.putText(
            combined,
            name,
            (idx * target_width + 10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (255, 255, 255),
            2,
        )

    return combined


def get_hash(x, y, w, h, cx, cy):
    data = f"{x}, {y}, {w}, {h}, {cx}, {cy}"
    d_bytes = data.encode("utf-8")
    d_hash = hashlib.sha512(d_bytes).hexdigest()
    return d_hash

In [ ]:
df = []
cap = cv2.VideoCapture(1)
while True:
    mean_data = {"x": [], "y": [], "w": [], "h": [], "cx": [], "cy": []}
    ret, frame = cap.read()
    if not ret:
        print("Failed!")
        break

    gr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    conv_frame = cv2.threshold(gr, 210, 255, cv2.THRESH_BINARY)[1]
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        conv_frame, 12, cv2.CV_32S
    )

    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cx, cy = int(centroids[i][0]), int(centroids[i][1])
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

        mean_data["x"].append(x)
        mean_data["y"].append(y)
        mean_data["w"].append(w)
        mean_data["h"].append(h)
        mean_data["cx"].append(cx)
        mean_data["cy"].append(cy)

    combined = create_combined_window(
        {"Original": frame, "Binary": conv_frame},
        target_width=640,  # Adjust this value to change the size of each window
    )

    mean_frame = pd.DataFrame(mean_data)
    hash = get_hash(*mean_frame.mean().values)
    df.append(mean_frame.mean().tolist())
    df[-1].append(hash)
    # Show single window with both frames
    cv2.imshow("Lava Lamp Detection", combined)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
display(pd.DataFrame(df, columns=mean_frame.columns.tolist() + ["hash"]))

,x,y,w,h,cx,cy,hash
0,627.752809,370.003745,6.842697,7.580524,630.400749,373.044944,e9ef5d9c780c65441c4cf8fb3f07ab913e834dff0c36d0...
1,637.963100,371.276753,7.044280,7.977860,640.686347,374.498155,45df49533a6464efc01cfda7f5beb0c522ff54d600ece3...
2,650.922222,370.637037,7.318519,8.311111,653.755556,373.985185,2b738ca3134a317f5ab0f0f59a54ce184778bfb162e6d5...
3,642.892734,374.768166,7.103806,8.086505,645.633218,377.996540,74bb0084852f9a2357a890d72c3c52b8ab939ca2bc7111...
4,635.643939,365.359848,7.242424,7.837121,638.477273,368.484848,62db3491bf2bb75295bf282e65b6e9f034c879c9935320...
...,...,...,...,...,...,...,...
138,607.003953,408.193676,8.233202,7.992095,610.304348,411.079051,cf1461805f1a8db0056c010404a69af227e63f8ad9293b...
139,603.876923,403.619231,7.996154,8.061538,607.100000,406.519231,2a5046b152d655d7cad541962865443e0f988db94060fd...
140,615.738636,412.409091,8.143939,7.958333,618.996212,415.253788,e0bdb44128d16629ea2f9ec251b6e8fc078b226c9f821d...
141,607.670683,406.947791,7.325301,7.963855,610.538153,409.831325,b2de47144794013e6ba3b85b7612411d6221db110c4db3...


In [16]:
df = pd.DataFrame(df, columns=mean_frame.columns.tolist() + ["hash"])
df.to_csv("lava_lamp.csv", index=False)